# Solving elasticity problems using JuliaFEM

Author(s): Jukka Aho

**Abstract**: A workflow to solve typical elasticity problem. This document also tries to give some quidelines how to develop JuliaFEM.

## Bottom-up design

We go piece by piece starting from something simple and going up to more complicated programming model.

*Design principle 1*: we introduce new ideas using Notebooks.

*Design principle 2*: we use ``Logging``. Forget ``println``.

In [1]:
using Logging
using ForwardDiff
Logging.configure(level=DEBUG)

Logger(root,DEBUG,Pipe(open, 0 bytes waiting),root)

*Design principle 3*: we write docstrings using [numpy style](https://github.com/numpy/numpy/blob/master/doc/HOWTO_DOCUMENT.rst.txt).

*Design principle 4*: we don't use greek characters in code which is implemented to JuliaFEM. In notebooks they are ok.

*Design principle 5*: we use 4 space indentation like in Python.

First we write some elementary functions to calculate stiffness matrix.

In [18]:
"""
Calculate local tangent stiffness matrix and residual force vector
R = T - F for elasticity problem.

Parameters
----------
X : Element coordinates
u : Displacement field
R : Residual force vector
K : Tangent stiffness matrix
basis : Basis functions
dbasis : Derivative of basis functions
lambda : Material parameter
mu : Material parameter
ipoints : integration points
iweights : integration weights

Returns
-------
None

Notes
-----
If material parameters are given in list, they are interpolated to gauss
points using shape functions.

Examples
--------

"""
function calc_local_matrices2!(X, u, R, K, basis, dbasis, lambda_, mu_, ipoints, iweights)
    dim, nnodes = size(X)
    I = eye(dim)
    R[:,:] = 0.0
    K[:,:] = 0.0

    dF = zeros(dim, dim)

    for m = 1:length(iweights)
        w = iweights[m]
        xi = ipoints[m, :]
        # calculate material parameters
        lambda = typeof(lambda_) == Float64 ? lambda_ : dot(lambda_, basis(xi))
        mu = typeof(mu_) == Float64 ? mu_ : dot(mu_, basis(xi))
        Jt = X*dbasis(xi)
        detJ = det(Jt)
        dbasisdX = dbasis(xi)*inv(Jt)

        gradu = u*dbasisdX
        F = I + gradu  # Deformation gradient
        E = 1/2*(gradu' + gradu + gradu'*gradu)  # Green-Lagrange strain tensor
        S = lambda*trace(E)*I + 2*mu*E  # PK2 stress tensor
        P = F*S  # PK1 stress tensor

        R[:,:] += w*P*dbasisdX'*detJ

        for p = 1:nnodes
            for i = 1:dim
                dF[:,:] = 0.0
                dF[i,:] = dbasisdX[p,:]
                dE = 1/2*(F'*dF + dF'*F)
                dS = lambda*trace(dE)*I + 2*mu*dE
                dP = dF*S + F*dS
                for q = 1:nnodes
                    for j = 1:dim
                        K[dim*(p-1)+i,dim*(q-1)+j] += w*(dP[j,:]*dbasisdX[q,:]')[1]*detJ
                    end
                end
            end
        end

    end
end

"""
Autodiff version.
"""
function calc_local_matrices!(X, u, R, K, basis, dbasis, lambda_, mu_, ipoints, iweights)
    dim, nnodes = size(X)
    I = eye(dim)
    R[:,:] = 0.0

    #dF = zeros(dim, dim)

    function calc_R!(u, R)
        for m = 1:length(iweights)
            w = iweights[m]
            xi = ipoints[m, :]
            # calculate material parameters
            lambda = typeof(lambda_) == Float64 ? lambda_ : dot(lambda_, basis(xi))
            mu = typeof(mu_) == Float64 ? mu_ : dot(mu_, basis(xi))
            Jt = X*dbasis(xi)
            detJ = det(Jt)
            dbasisdX = dbasis(xi)*inv(Jt)

            gradu = u*dbasisdX
            F = I + gradu  # Deformation gradient
            E = 1/2*(gradu' + gradu + gradu'*gradu)  # Green-Lagrange strain tensor
            S = lambda*trace(E)*I + 2*mu*E  # PK2 stress tensor
            P = F*S  # PK1 stress tensor

            R[:,:] += w*P*dbasisdX'*detJ
        end
    end

    # herlper for tangent stiffness matrix
    function R!(u, R)
        R[:] = 0
        calc_R!(reshape(u, dim, nnodes), reshape(R, dim, nnodes))
        #calc_Wext!(reshape(u, 2, 4), reshape(R, 2, 4))
    end
    Jacobian = ForwardDiff.forwarddiff_jacobian(R!, Float64, fadtype=:dual, n=dim*nnodes, m=dim*nnodes)

    K[:, :] = Jacobian(reshape(u, dim*nnodes))
    R!(reshape(u, dim*nnodes), reshape(R, dim*nnodes))

end

calc_local_matrices! (generic function with 1 method)

*Design principle 6*: we test our code. We use FactCheck for testing.

In [16]:
using FactCheck

In [19]:
facts("test solve one element model") do
    X = [0.0 0.0; 10.0 0.0; 10.0 1.0; 0.0 1.0]'
    F = [0 0; 0 0; 0 -2; 0 0]'

    # Material properties
    E = 90
    nu = 0.25
    mu = E/(2*(1+nu))
    la = E*nu/((1+nu)*(1-2*nu))
    la = 2*la*mu/(la + 2*mu)

    u = zeros(2, 4)
    du = zeros(2, 4)
    R = zeros(2, 4)
    K = zeros(8, 8)

    basis(xi) = [
        (1-xi[1])*(1-xi[2])/4
        (1+xi[1])*(1-xi[2])/4
        (1+xi[1])*(1+xi[2])/4
        (1-xi[1])*(1+xi[2])/4]

    dbasis(xi) = [-(1-xi[2])/4.0    -(1-xi[1])/4.0
                   (1-xi[2])/4.0    -(1+xi[1])/4.0
                   (1+xi[2])/4.0     (1+xi[1])/4.0
                  -(1+xi[2])/4.0     (1-xi[1])/4.0]

    ipoints = 1/sqrt(3)*[-1 -1; 1 -1; 1 1; -1 1]
    iweights = [1, 1, 1, 1]
    free_dofs = [3, 4, 5, 6]

    for i=1:10
        calc_local_matrices!(X, u, R, K, basis, dbasis, la, mu, ipoints, iweights)
        du[free_dofs] = K[free_dofs, free_dofs] \ -(R - F)[free_dofs]
        u += du
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end

    # Tested against Elmer solution
    Logging.debug("solution vector: \n $u")
    @fact u[2, 3] --> roughly(-2.222244754401764)
    norm1 = norm(u)
    Logging.debug("norm of u: $(norm(u))")

    # We rotate model a bit and make sure that L2 norm is same
    phi = 30/180*pi
    rmat = [
        cos(phi) -sin(phi)
        sin(phi)  cos(phi)]
    X = rmat*X
    F = rmat*F
    u = zeros(2, 4)
    for i=1:10
        calc_local_matrices!(X, u, R, K, basis, dbasis, la, mu, ipoints, iweights)
        du[free_dofs] = K[free_dofs, free_dofs] \ -(R - F)[free_dofs]
        u += du
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end
    Logging.debug("solution vector: \n $u")
    Logging.debug("norm of u: $(norm(u))")
    @fact norm(u) --> roughly(norm1) 

    # test two element model
    X = [0.0 0.0; 5.0 0.0; 5.0 1.0; 0.0 1.0]'
    u = zeros(2, 6)
    du = zeros(2, 6)
    R = zeros(2, 4)
    K = zeros(8, 8)
    ass1 = [9, 10, 1, 2, 5, 6, 11, 12]
    ass2 = [1, 2, 3, 4, 7, 8, 5, 6]
    free_dofs = collect(1:8)
    F = [0 0; 0 0; 0 0; 0 -0.1; 0 0; 0 0]'

    A = zeros(12, 12)
    b = zeros(2, 6)
    for i=1:1
        Logging.debug("Iteration $i")
        A[:,:] = 0.0
        b[:] = 0.0
        #Logging.debug("Assembling")
        for ass in (ass1, ass2)
            #Logging.debug("ass = $ass, u[ass] = $(u[ass])")
            calc_local_matrices!(X, u[ass], R, K, basis, dbasis, la, mu, ipoints, iweights)
            A[ass,ass] += K
            b[ass] += R[:]
        end
        dump(round(A, 2))
        println("K norm = $(norm(A[free_dofs, free_dofs]))")
        du[free_dofs] = A[free_dofs, free_dofs] \ -(b - F)[free_dofs]
        println("du = $du")
        u += du
        Logging.debug("Norm of du: $(norm(du))")
        for ass in (ass1, ass2)
            Logging.debug("Element displacement: $(reshape(u[ass], 2, 4))")
        end
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end
    Logging.debug("solution vector: \n $u")
    Logging.debug("norm of u: $(norm(u))")
    @pending norm(u) --> :something
end

test solve one element model


11-Aug 21:42:03:DEBUG:root:Converged in 6 iterations.
11-Aug 21:42:03:DEBUG:root:solution vector: 
 [0.0 -0.3991450609547433 -0.07228582695592461 0.0
 0.0 -2.1779892317073504 -2.222244754401764 0.0]
11-Aug 21:42:03:DEBUG:root:norm of u: 3.1292483947150047
11-Aug 21:42:03:DEBUG:root:Converged in 6 iterations.
11-Aug 21:42:03:DEBUG:root:solution vector: 
 [0.0 0.7433248532717796 1.048521014723486 0.0
 0.0 -2.085766534304891 -1.9606633242166027 0.0]
11-Aug 21:42:03:DEBUG:root:norm of u: 3.1292483947150056
11-Aug 21:42:03:DEBUG:root:Iteration 1


Array(Float64,(12,12)) 12x12 Array{Float64,2}

11-Aug 21:42:04:DEBUG:root:Norm of du: 0.5992228342549063
11-Aug 21:42:04:DEBUG:root:Element displacement: [0.0 -0.02264423092574128 0.022536491965822688 0.0
 0.0 -0.12688379170176511 -0.12679760053383046 0.0]
11-Aug 21:42:04:DEBUG:root:Element displacement: [-0.02264423092574128 -0.029998807330416523 0.030242156525002267 0.022536491965822688
 -0.12688379170176511 -0.4041002710283739 -0.40446733271991214 -0.12679760053383046]
11-Aug 21:42:04:DEBUG:root:solution vector: 
 [-0.02264423092574128 -0.029998807330416523 0.022536491965822688 0.030242156525002267 0.0 0.0
 -0.12688379170176511 -0.4041002710283739 -0.12679760053383046 -0.40446733271991214 0.0 0.0]
11-Aug 21:42:04:DEBUG:root:norm of u: 0.5992228342549063


delayed_handler (generic function with 4 methods)

:
  132.8     0.0   23.6    -3.0  -113.6     0.0  -33.2   -15.0   23.6     3.0  -33.2    15.0
    0.0   324.8    3.0    77.6     0.0  -317.6  -15.0   -81.2   -3.0    77.6   15.0   -81.2
   23.6     3.0   66.4   -15.0   -33.2    15.0  -56.8    -3.0    0.0     0.0    0.0     0.0
   -3.0    77.6  -15.0   162.4    15.0   -81.2    3.0  -158.8    0.0     0.0    0.0     0.0
 -113.6     0.0  -33.2    15.0   132.8     0.0   23.6     3.0  -33.2   -15.0   23.6    -3.0
    0.0  -317.6   15.0   -81.2     0.0   324.8   -3.0    77.6  -15.0   -81.2    3.0    77.6
  -33.2   -15.0  -56.8     3.0    23.6    -3.0   66.4    15.0    0.0     0.0    0.0     0.0
  -15.0   -81.2   -3.0  -158.8     3.0    77.6   15.0   162.4    0.0     0.0    0.0     0.0
   23.6    -3.0    0.0     0.0   -33.2   -15.0    0.0     0.0   66.4    15.0  -56.8     3.0
    3.0    77.6    0.0     0.0   -15.0   -81.2    0.0     0.0   15.0   162.4   -3.0  -158.8
  -33.2    15.0    0.0     0.0    23.6     3.0    0.0     0.0  -56.8    -3.0  

One element solutions are not particularly interesting so next step is to create function that assembles global matrix from local matrices. Some data types:

In [5]:
#type Node
#    id :: Int
#    #elements :: Array{Int64, 1}
#end

In [6]:
type Element
    id :: Int
    node_ids :: Array{Int64, 1}
    coordinates :: Array{Float64, 2}
    attributes :: Dict{ASCIIString, Any}
end

In [7]:
type Assembly
    # LHS
    I :: Array{Int64, 1}
    J :: Array{Int64, 1}
    A :: Array{Float64, 1}
    # RHS
    i :: Array{Int64, 1}
    b :: Array{Float64, 1}
    # global dofs for each element
    gdofs :: Dict{Int64, Array{Int64, 1}}
end

In [8]:
"""
Return shape functions and their derivatives for a element.

Parameters
----------
element::Element

Returns
-------
tuple (basis, dbasis)
"""
function get_shape_functions(el::Element)
    ndim, nnodes = size(el.coordinates)
    #Logging.debug("ndim = $ndim, nnodes=$nnodes")
    if (nnodes == 4) & (ndim == 2)
        basis(xi) = [
            (1-xi[1])*(1-xi[2])/4
            (1+xi[1])*(1-xi[2])/4
            (1+xi[1])*(1+xi[2])/4
            (1-xi[1])*(1+xi[2])/4]
        dbasis(xi) = [-(1-xi[2])/4.0    -(1-xi[1])/4.0
                       (1-xi[2])/4.0    -(1+xi[1])/4.0
                       (1+xi[2])/4.0     (1+xi[1])/4.0
                      -(1+xi[2])/4.0     (1-xi[1])/4.0]
        return basis, dbasis
    elseif (nnodes == 10) & (ndim == 3)
        basis(xi) = [(xi[1] + xi[2] + xi[3] - 1)*(2*xi[1] + 2*xi[2] + 2*xi[3] - 1)
                                        -xi[1]*(-2*xi[1] + 1)
                                        -xi[2]*(-2*xi[2] + 1)
                                        -xi[3]*(-2*xi[3] + 1)
                         4*xi[1]*(-xi[1] - xi[2] - xi[3] + 1)
                                                4*xi[1]*xi[2]
                         4*xi[2]*(-xi[1] - xi[2] - xi[3] + 1)
                                                4*xi[1]*xi[3]
                                                4*xi[2]*xi[3]
                         4*xi[3]*(-xi[1] - xi[2] - xi[3] + 1)]

        dbasis(xi) = [
            4*xi[1] + 4*xi[2] + 4*xi[3] - 3   4*xi[1] + 4*xi[2] + 4*xi[3] - 3   4*xi[1] + 4*xi[2] + 4*xi[3] - 3
                             4*xi[1] - 1                                0                                0
                                       0                      4*xi[2] - 1                                0
                                       0                                0                      4*xi[3] - 1
        -8*xi[1] - 4*xi[2] - 4*xi[3] + 4                         -4*xi[1]                         -4*xi[1]
                                 4*xi[2]                          4*xi[1]                                0
                                -4*xi[2]  -4*xi[1] - 8*xi[2] - 4*xi[3] + 4                         -4*xi[2]
                                 4*xi[3]                                0                          4*xi[1]
                                       0                          4*xi[3]                          4*xi[2]
                                -4*xi[3]                         -4*xi[3] -4*xi[1] - 4*xi[2] - 8*xi[3] + 4]
        return basis, dbasis
    end
    throw("Unknown function space, ndim=$ndim, nnodes=$nnodes")
end

"""
"""
function get_integration_scheme(el::Element, order=2)
    ndim, nnodes = size(el.coordinates)
    if (nnodes == 4) & (order == 2) & (ndim == 2)
        ipoints = 1/sqrt(3)*[-1 -1; 1 -1; 1 1; -1 1]
        iweights = [1, 1, 1, 1]
        return ipoints, iweights
    elseif (nnodes == 10) & (order == 2) & (ndim == 3) # c3d10
        # from code aster documentation
        a = 1/20*(5-sqrt(5))
        b = 1/20*(5+3*sqrt(5))
        ipoints = [a a a; a a b; a b a; b a a]
        iweights = 1/24*[1 1 1 1]
        return ipoints, iweights
    end
end

get_integration_scheme (generic function with 2 methods)

In [9]:
function assemble_element!(ass::Assembly, el::Element, io=2)

    # Material properties
    E = el.attributes["Young"]
    nu = el.attributes["Poisson"]
    mu = E/(2*(1+nu))
    la = E*nu/((1+nu)*(1-2*nu))
    la = 2*la*mu/(la + 2*mu)

    dofs = prod(size(el.coordinates))
    X = el.coordinates
    u = el.attributes["displacement"]
    R = el.attributes["displacement nodal force"]
    K = el.attributes["displacement tangent stiffness"]

    gdofs = ass.gdofs[el.id]
    #Logging.debug("Assemble element $(el.id) to gdofs $gdofs")
    basis, dbasis = get_shape_functions(el)
    ipoints, iweights = get_integration_scheme(el, io)
    calc_local_matrices!(X, u, R, K, basis, dbasis, la, mu, ipoints, iweights)

    for i=1:dofs
        for j=1:dofs
            push!(ass.I, gdofs[i])
            push!(ass.J, gdofs[j])
            push!(ass.A, K[i,j])
        end
        push!(ass.i, gdofs[i])
        push!(ass.b, R[i])
    end
end

assemble_element! (generic function with 2 methods)

Time to test again. From last test we know that correct solution is

    [0.0 -0.39914506095474317 -0.07228582695592449 0.0
     0.0 -2.1779892317073504  -2.222244754401764   0.0]

This time we assemble global stiffness matrix in different order, 2 3 4 1

In [10]:
facts("one element assembly") do
    # Create model
    #Logging.debug("Creating nodes")
    #n1 = Node(1)
    #n2 = Node(2)
    #n3 = Node(3)
    #n4 = Node(4)
    Logging.debug("Adding nodes to array")
    #nodes = [n1.id, n2.id, n3.id, n4.id]
    node_ids = [1, 2, 3, 4]
    coordinates = [10.0 0.0; 10.0 1.0; 0.0 1.0; 0.0 0.0]'
    attributes = Dict("Young" => 90, "Poisson" => 0.25)
    Logging.debug("Creating elements")
    el = Element(1, node_ids, coordinates, attributes)

    # Initialize elements ready for solution
    el.attributes["displacement"] = zeros(2, 4)
    el.attributes["displacement nodal force"] = zeros(2, 4)
    el.attributes["displacement tangent stiffness"] = zeros(8, 8)

    for i=1:10
        Logging.debug("Starting iteration $i")
        ass = Assembly(Int64[], Int64[], Float64[], Int64[], Float64[], Dict{Int64,Array{Int64,1}}())
        ass.gdofs[el.id] = [1, 2, 3, 4, 5, 6, 7, 8]
        Logging.debug("Assembling")
        assemble_element!(ass, el)

        # Boundary conditions
        F = [0 0; 0 -2; 0 0; 0 0]'
        F = reshape(F, prod(size(F)))
        free_dofs = [1, 2, 3, 4]

        # solution
        K = sparse(ass.I, ass.J, ass.A)
        R = full(sparsevec(ass.i, ass.b))
        R = R - F
        du = zeros(8) # must be determined from ass
        du[free_dofs] = K[free_dofs, free_dofs] \ -R[free_dofs]

        Logging.debug("Solution norm = $(norm(du))")

        # update solution back to elements
        eldu = du[ass.gdofs[el.id]]
        eldu = reshape(eldu, (2, round(Int, length(eldu)/2)))
        el.attributes["displacement"] += eldu
        if norm(du) < 1.0e-9
            Logging.debug("Converged in $i iterations.")
            break
        end
    end
    disp = el.attributes["displacement"]
    Logging.debug("Displacement of element = \n$disp")
    @fact norm(disp) => roughly(3.1292483947150043)
end

one element assembly


11-Aug 21:39:42:DEBUG:root:Adding nodes to array
11-Aug 21:39:42:DEBUG:root:Creating elements
11-Aug 21:39:42:DEBUG:root:Starting iteration 1
11-Aug 21:39:42:DEBUG:root:Assembling
11-Aug 21:39:42:DEBUG:root:Solution norm = 3.090022136728999
11-Aug 21:39:42:DEBUG:root:Starting iteration 2
11-Aug 21:39:42:DEBUG:root:Assembling
11-Aug 21:39:43:DEBUG:root:Solution norm = 0.32121316021535135
11-Aug 21:39:43:DEBUG:root:Starting iteration 3
11-Aug 21:39:43:DEBUG:root:Assembling
11-Aug 21:39:43:DEBUG:root:Solution norm = 0.040431781939994194
11-Aug 21:39:43:DEBUG:root:Starting iteration 4
11-Aug 21:39:43:DEBUG:root:Assembling
11-Aug 21:39:43:DEBUG:root:Solution norm = 0.0009291101052124042
11-Aug 21:39:43:DEBUG:root:Starting iteration 5
11-Aug 21:39:43:DEBUG:root:Assembling
11-Aug 21:39:43:DEBUG:root:Solution norm = 1.5638899022213175e-7
11-Aug 21:39:43:DEBUG:root:Starting iteration 6
11-Aug 21:39:43:DEBUG:root:Assembling
11-Aug 21:39:43:DEBUG:root:Solution norm = 1.0118539067290854e-14
11-Aug

1 fact verified.


delayed_handler (generic function with 4 methods)

Seems to be working. But we still need to handle boundary conditions more "cleverly" and generalize assembly to several elements (which is not problem).

First of all, essential boundary conditions are nothing more than equality constraints saying that value for some degree of freedom is fixed. Elimination is just a special case when this value equals to zero. There is couple of different strategies to handle essential boundary conditions. One option is to force them using Lagrange multipliers which can also be used to create all kind of kinematic constraints also. (For example, contact can be considered as a kinematic constraint.) Another option is to manipulate matrix such a way that constraint is satisfied.

Because we are now going "bottom-up", we develop something extremely simple that however deals with the problem:

In [11]:
type BC
    dofs :: Array{Int64, 1}
    values :: Array{Float64, 1}
end

In [12]:
"""
Create local dof to global dof mapping for given elements
"""
function create_ldof2gdofmap(elements; ndofs=2)
    Logging.info("create_ldof2gdofmap: dofs per node: $ndofs")

    all_node_ids = Int64[]
    for el in elements
        eldim, elnodes = size(el.coordinates)
        for nid in el.node_ids
            push!(all_node_ids, nid)
        end
    end
    all_node_ids = unique(all_node_ids)
    sort!(all_node_ids)

    # Assign global dof for each node
    pdim = 1
    ngdofs = Dict{Int64, Array{Int64,1}}()
    for nid in all_node_ids
        ngdofs[nid] = collect(pdim:pdim+ndofs-1)
        pdim += ndofs
    end

    return ngdofs
end

function solve!(elements, dofmap, neumann_bcs, dirichlet_bcs; ndofs=2, max_iterations=10)

    dbc = "lagrange"
    
    Logging.info("solve!: dofs per node: $ndofs")
    pdim = length(dofmap)*ndofs
    Logging.debug("Problem size = $pdim")

    # Initialize elements ready for solution
    for el in elements
        eldim, elnodes = size(el.coordinates)
        el.attributes["displacement"] = zeros(ndofs, elnodes)
        el.attributes["displacement nodal force"] = zeros(ndofs, elnodes)
        el.attributes["displacement tangent stiffness"] = zeros(ndofs*elnodes, ndofs*elnodes)
    end

    # Assign global dofs for elements
    gdofs = Dict{Int64, Array{Int64,1}}()
    for el in elements
        gdofs[el.id] = Int64[]
        for nid in el.node_ids
            for ndof in dofmap[nid]
                push!(gdofs[el.id], ndof)
            end
        end
    end

    ass = Assembly(Int64[], Int64[], Float64[], Int64[], Float64[], gdofs)

    for iter=1:max_iterations
        Logging.debug("Starting iteration $iter")
        ass.I = []
        ass.J = []
        ass.A = []
        ass.i = []
        ass.b = []
        
        Logging.debug("Assembling")
        for el in elements
            assemble_element!(ass, el)
            #println("Element stiffness matrix")
            #dump(round(el.attributes["displacement tangent stiffness"], 2))
        end

        i = 1
        if dbc == "lagrange"
            Logging.debug("Adding Dirichlet boundary conditions using Lagrange multipliers")
            # Dirichlet boundary conditions
            for bc in dirichlet_bcs
                for (dof, val) in zip(bc.dofs, bc.values)
                    #Logging.debug("dof $dof => $val")
                    push!(ass.I, dof)
                    push!(ass.J, pdim+i)
                    push!(ass.A, 1)
                    push!(ass.I, pdim+i)
                    push!(ass.J, dof)
                    push!(ass.A, 1)
                    push!(ass.i, pdim+i)
                    push!(ass.b, 0)
                    i += 1
                end
            end
            Logging.debug("Added $i Lagrange multipliers to model")
        end
        i -= 1

        Logging.debug("Adding Neumann boundary conditions")
        F = zeros(pdim+i)
        # Neumann boundary conditions
        for bc in neumann_bcs
            for (dof, val) in zip(bc.dofs, bc.values)
                F[dof] += val
            end
        end

        Logging.debug("Solving system of equations. Total size = $(pdim+i)")
        # solution
        K = sparse(ass.I, ass.J, ass.A)
        R = full(sparsevec(ass.i, ass.b))
        R = R - F
        #Logging.debug(dump(round(full(K), 2)))
        #print_matrix(full(K))
        #Logging.debug(dump(round(R', 1)))

        if dbc == "eliminate"
            Logging.debug("Eliminating Dirichlet boundary conditions")
            throw("Implement this properly")
            free_dofs = [1, 2, 3, 4, 5, 6, 7, 8]
            du = zeros(12)
            Logging.debug("K norm = $(norm(full(K[free_dofs, free_dofs])))")
            du[free_dofs] = K[free_dofs, free_dofs] \ -R[free_dofs]
        else
            du = K \ -R
        end

        #du = reshape(du, 2, 6)
        solnorm = norm(du[1:pdim])
        #Logging.debug("du = $du")
        Logging.debug("Solution norm du = $solnorm")

        # update solution back to elements
        for el in elements
            #Logging.debug("update element $(el.id)")
            eldisp = el.attributes["displacement"]
            #Logging.debug("displacement before update $(el.id) : \n$eldisp")
            eldu = du[ass.gdofs[el.id]]
            eldu = reshape(eldu, (ndofs, round(Int, length(eldu)/ndofs)))
            #Logging.debug("eldu for element $(el.id) \n$eldu")
            el.attributes["displacement"] += eldu
            eldisp = el.attributes["displacement"]
            #Logging.debug("displacement after update $(el.id) : \n$eldisp")

        end
        if solnorm < 1.0e-9
            Logging.debug("Converged in $iter iterations.")
            break
        end
    end

end

ENV["COLUMNS"] = 160

function test1():
    facts("solve one element problem") do
        # Create model
        Logging.debug("Creating nodes")
        node_ids = [1, 2, 3, 4]
        coordinates = [10.0 0.0; 10.0 1.0; 0.0 1.0; 0.0 0.0]'
        attributes = Dict("Young" => 90, "Poisson" => 0.25)
        Logging.debug("Creating elements")
        el = Element(1, node_ids, coordinates, attributes)
        elements = [el]
        dofmap = create_ldof2gdofmap(elements)
        Logging.debug(dofmap)
        # Boundary conditions
        # here we want to create nodal force for third dof, that is, node id 2, second dof
        bc1 = BC([dofmap[2][2]], [-2.0])
        # dirichlet bc, set dx=dy=0 on support
        bc2 = BC([dofmap[3][1], dofmap[3][2], dofmap[4][1], dofmap[4][2]], [0.0, 0.0, 0.0, 0.0])
        solve!(elements, dofmap, [bc1], [bc2]; max_iterations=7)
        disp = elements[1].attributes["displacement"]
        Logging.debug("Displacement of element = \n$disp")
        @fact norm(disp) => roughly(3.1292483947150043)
    end
end

facts("solve two element problem") do
    # Create model
    attributes = Dict("Young" => 90, "Poisson" => 0.25)

    Logging.debug("Creating elements")
    nids1 = [5, 1, 3, 6]
    coords1 = [0.0 0.0; 5.0 0.0; 5.0 1.0; 0.0 1.0]'
    el1 = Element(1, nids1, coords1, copy(attributes))
    nids2 = [1, 2, 4, 3]
    coords2 = [5.0 0.0; 10.0 0.0; 10.0 1.0; 5.0 1.0]'
    el2 = Element(2, nids2, coords2, copy(attributes))
    elements = [el1, el2]

    dofmap = create_ldof2gdofmap(elements)
    Logging.debug(dofmap)
    # Boundary conditions
    # here we want to create nodal force for third dof, that is, node id 2, second dof
    bc1 = BC([dofmap[4][2]], [-0.1])
    # dirichlet bc, set dx=dy=0 on support
    bc2 = BC([dofmap[5][1], dofmap[5][2], dofmap[6][1], dofmap[6][2]], [0.0, 0.0, 0.0, 0.0])
    solve!(elements, dofmap, [bc1], [bc2]; max_iterations=7)
    disp = elements[2].attributes["displacement"]
    Logging.debug("Displacement of element = \n$disp")
    #@fact norm(disp) => roughly(3.1292483947150043)
end

solve two element problem


11-Aug 21:39:49:DEBUG:root:Creating elements
11-Aug 21:39:49:INFO:root:create_ldof2gdofmap: dofs per node: 2
11-Aug 21:39:49:DEBUG:root:Dict(4=>[7,8],2=>[3,4],3=>[5,6],5=>[9,10],6=>[11,12],1=>[1,2])
11-Aug 21:39:50:INFO:root:solve!: dofs per node: 2
11-Aug 21:39:50:DEBUG:root:Problem size = 12
11-Aug 21:39:50:DEBUG:root:Starting iteration 1
11-Aug 21:39:50:DEBUG:root:Assembling
11-Aug 21:39:50:DEBUG:root:Adding Dirichlet boundary conditions using Lagrange multipliers
11-Aug 21:39:50:DEBUG:root:Added 5 Lagrange multipliers to model
11-Aug 21:39:50:DEBUG:root:Adding Neumann boundary conditions
11-Aug 21:39:50:DEBUG:root:Solving system of equations. Total size = 16
11-Aug 21:39:51:DEBUG:root:Solution norm du = 0.6015838690633517
11-Aug 21:39:51:DEBUG:root:Starting iteration 2
11-Aug 21:39:51:DEBUG:root:Assembling
11-Aug 21:39:51:DEBUG:root:Adding Dirichlet boundary conditions using Lagrange multipliers
11-Aug 21:39:51:DEBUG:root:Added 5 Lagrange multipliers to model
11-Aug 21:39:51:DEBUG:

0 facts verified.


delayed_handler (generic function with 4 methods)

## Top-down design

Next we see this problem from "other direction", by parsing ABAQUS .inp file and making 3d simulation.

In [13]:
using JuliaFEM.abaqus_reader
fid = open("../geometry/3d_beam/palkki.inp")
model = JuliaFEM.abaqus_reader.parse_abaqus(fid)
close(fid)
model


Use "Dict{Any,Any}(a=>b, ...)" instead.

Use "Dict{Any,Any}(a=>b, ...)" instead.
11-Aug 21:40:01:INFO:root:Registered handlers: Any["ELEMENT","NODE","NSET"]
 in depwarn at /Applications/Julia-0.4.0-dev-539c818c4e.app/Contents/Resources/julia/lib/julia/sys.dylib
 in beginswith at deprecated.jl:30
 in parse_abaqus at /Users/jukka/.julia/v0.4/JuliaFEM/src/abaqus_reader.jl:113
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at /Users/jukka/.julia/v0.4/IJulia/src/execute_request.jl:157
 in eventloop at /Users/jukka/.julia/v0.4/IJulia/src/IJulia.jl:123
 in anonymous at task.jl:365
while loading In[13], in expression starting on line 3
 in depwarn at /Applications/Julia-0.4.0-dev-539c818c4e.app/Contents/Resources/julia/lib/julia/sys.dylib
 in beginswith at deprecated.jl:30
 in parse_abaqus at /Users/jukka/.julia/v0.4/JuliaFEM/src/abaqus_reader.jl:113
 in include_string at loading.jl:99
 in execute_request_0x535c5df2 at /Users/jukka/.julia/v0.4/IJulia/src/execute_request.jl

Dict{Any,Any} with 4 entries:
  "nodes"    => Dict{Any,Any}(288=>[97.5,7.5,10.0],11=>[92.5,2.5,5.0],134=>[45.0,10.0,0.0],158=>[2.5,2.5,0.0],160=>[7.5,7.5,0.0],215=>[60.0,0.0,5.0],29=>[2.5,7…
  "elements" => Dict{Any,Any}(68=>[71,144,149,198,51,150,57,43,50,214],2=>[204,199,175,130,207,208,209,3,4,176],89=>[95,78,104,52,127,126,106,60,68,67],11=>[15…
  "elsets"   => Dict{Any,Any}("Body1"=>[1,2,3,4,5,6,7,8,9,10  …  111,112,113,114,115,116,117,118,119,120])
  "nsets"    => Dict{Any,Any}("LOAD"=>[82,84,87,179,197,246,249,256,257],"SUPPORT"=>[108,109,111,155,162,216,225,281,298],"TOP"=>[70,75,76,84,88,90,95,96,98,10…

In [34]:
function solve_3d_model()
    Logging.debug("Creating elements")
    elements = Element[]
    coordinates = zeros(3, 10)
    for (elid, node_ids) in model["elements"]
        for (i, nid) in enumerate(node_ids)
            coordinates[:,i] = model["nodes"][nid]
        end
        attributes = Dict("Young" => 90, "Poisson" => 0.25)
        el = Element(elid, node_ids, coordinates, attributes)
        push!(elements, el)
    end

    # create "dofmap" so that we know how to assemble global stiffness matrix
    dofmap = create_ldof2gdofmap(elements; ndofs=3)

    # Boundary conditions

    # dirichlet bc, set dx=dy=dz for all nodes in set SUPPORT
    bc_support = BC(Int64[], Float64[])
    for nid in model["nsets"]["SUPPORT"]
        for i=1:3
            push!(bc_support.dofs, dofmap[nid][i])
            push!(bc_support.values, 0.0)
        end
    end

    # force boundary condition, put -1 to 2nd dof for each node in set LOAD
    bc_load = BC(Int64[], Float64[])
    for nid in model["nsets"]["LOAD"]
        push!(bc_load.dofs, dofmap[nid][2])
        push!(bc_load.values, -30.0)
    end

    #solve!(elements, [bc1], [bc2]; max_iterations=7)
    neumann_bcs = [bc_load]
    dirichlet_bcs = [bc_support]
    # ndofs = dimension of unknown field in nodes
    solve!(elements, dofmap, neumann_bcs, dirichlet_bcs; ndofs=3, max_iterations=10)

    # Let's pick maximum absolute displacement in y direction
    maxdisp = 0.0
    for el in elements
        eldisp = el.attributes["displacement"]
        eldispy = eldisp[2,:]
        maxeldisp = maximum(abs(eldispy))
        if maxeldisp > maxdisp
            maxdisp = maxeldisp
        end
    end
    Logging.info("Maximum absolute displacement in y direction: $maxdisp")
    return model, elements, dofmap
end

model, elements, dofmap = solve_3d_model();

11-Aug 22:15:29:DEBUG:root:Creating elements
11-Aug 22:15:29:INFO:root:create_ldof2gdofmap: dofs per node: 3
11-Aug 22:15:29:INFO:root:solve!: dofs per node: 3
11-Aug 22:15:29:DEBUG:root:Problem size = 894
11-Aug 22:15:29:DEBUG:root:Starting iteration 1
11-Aug 22:15:29:DEBUG:root:Assembling
11-Aug 22:15:30:DEBUG:root:Adding Dirichlet boundary conditions using Lagrange multipliers
11-Aug 22:15:30:DEBUG:root:Added 28 Lagrange multipliers to model
11-Aug 22:15:30:DEBUG:root:Adding Neumann boundary conditions
11-Aug 22:15:30:DEBUG:root:Solving system of equations. Total size = 921
11-Aug 22:15:30:DEBUG:root:Solution norm du = 72.87727091053921
11-Aug 22:15:30:DEBUG:root:Starting iteration 2
11-Aug 22:15:31:DEBUG:root:Assembling
11-Aug 22:15:32:DEBUG:root:Adding Dirichlet boundary conditions using Lagrange multipliers
11-Aug 22:15:32:DEBUG:root:Added 28 Lagrange multipliers to model
11-Aug 22:15:32:DEBUG:root:Adding Neumann boundary conditions
11-Aug 22:15:32:DEBUG:root:Solving system of eq

## Saving results to file

In [49]:
xdoc, xmodel = JuliaFEM.xdmf.xdmf_new_model()
temporal_collection = JuliaFEM.xdmf.xdmf_new_temporal_collection(xmodel)
grid = JuliaFEM.xdmf.xdmf_new_grid(temporal_collection; time=0)

<Grid Name="Grid">
  <Time Value="0"/>
</Grid>


Save geometry to xdmf file

In [50]:
nnodes = length(model["nodes"])
Logging.info("Number of nodes in model: $nnodes")
node_ids = Int64[]
for nid in keys(model["nodes"])
    push!(node_ids, nid)
end
sort!(node_ids)
X = zeros(3, nnodes)
for (i, nid) in enumerate(node_ids)
    X[:,i] = model["nodes"][nid]
end

nelements = length(model["elements"])
Logging.info("Number of elements in model: $nelements")
elmap = zeros(Int64, 11, nelements)
elmap[1,:] = 0x0026
for elid in 1:nelements
    elmap[2:end,elid] = model["elements"][elid]
end

11-Aug 22:38:28:INFO:root:Number of nodes in model: 298
11-Aug 22:38:28:INFO:root:Number of elements in model: 120


In [51]:
using LightXML

function xdmf_new_mesh(grid, X, elmap)
    dim, nnodes = size(X)
    geometry = new_child(grid, "Geometry")
    set_attribute(geometry, "Type", "XYZ")
    dataitem = new_child(geometry, "DataItem")
    set_attribute(dataitem, "DataType", "Float")
    set_attribute(dataitem, "Dimensions", "$nnodes $dim")
    set_attribute(dataitem, "Format", "XML")
    set_attribute(dataitem, "Precision", 8)
    #add_text(dataitem, join(X, " "))
    s = "\n"
    
    for i=1:nnodes
        s *= "\t\t" * join(X[:,i], " ") * "\n"
    end
    s *= "       "
    add_text(dataitem, s)

    elmap2 = copy(elmap)
    elmap2[2:end,:] -= 1
    dim, nelements = size(elmap2)

    topology = new_child(grid, "Topology")
    #set_attribute(topology, "Dimensions", "1")
    set_attribute(topology, "TopologyType", "Mixed")
    set_attribute(topology, "NumberOfElements", nelements)
    dataitem = new_child(topology, "DataItem")
    set_attribute(dataitem, "DataType", "Int")
    set_attribute(dataitem, "Dimensions", "$nelements $dim")
    set_attribute(dataitem, "Format", "XML")
    set_attribute(dataitem, "Precision", 8)
    s = "\n"
    for i=1:nelements
        s *= "\t\t" * join(elmap2[:,i], " ") * "\n"
    end
    add_text(dataitem, s)
    #add_text(dataitem, join(elmap2, " "))
    
end

xdmf_new_mesh(grid, X, elmap)

true

In [52]:
JuliaFEM.xdmf.xdmf_save_model(xdoc, "/tmp/3d_solid_model.xmf")

10435

Save nodal data to model

In [53]:
nodaldisp = Dict()
for el in elements
    for (i, nid) in enumerate(el.node_ids)
        nodaldisp[nid] = el.attributes["displacement"][:, i]
    end
end
nodaldisp

Dict{Any,Any} with 298 entries:
  288 => [-0.21784897897079505,-6.64256844921929,0.18339325827190953]
  11  => [-0.17342410820059667,-6.343125806991628,0.17544367431766225]
  158 => [-0.0007723334905839533,-0.2321904799732306,-0.03883691791045858]
  215 => [-0.03299227384130259,-4.2236276356937505,-0.0017757488281830807]
  134 => [-0.06336705102703495,-3.3332586850719337,0.06555932581531249]
  160 => [0.03205303832663507,-0.536361456950779,-0.07291283371223893]
  29  => [-0.01620524134284992,-0.19455583841608232,-0.03838421619811468]
  131 => [0.04015396763870081,-3.5101776818531523,0.10045744211179368]
  249 => [-0.2491004521409732,-6.570813533283954,0.06837890174506163]
  207 => [0.03974407267311282,-3.48588780058722,0.007971846403163977]
  173 => [-0.04472757590353555,-4.582185697159958,-0.04434348606400347]
  289 => [-0.16494613158592017,-6.360857515886982,0.33435404817411213]
  74  => [-0.03449870789750817,-3.947284664599636,0.0692691027146262]
  201 => [-0.00128347996939122,-3.85

In [54]:
u = zeros(3, nnodes)
for (i, nid) in enumerate(node_ids)
    u[:,i] = nodaldisp[nid]
end
u

3x298 Array{Float64,2}:
 -0.202475   -0.17437     0.0131267   -0.0340868  -0.0252162   0.0521758  …  -0.054781    -0.169903   0.186936   0.102666  -0.0970747  -0.164397  0.0
 -1.34204    -1.66404    -3.20587     -3.27673    -3.68732    -3.36789       -4.33822     -1.03943   -2.27466   -1.67467   -2.91213    -2.52353   0.0
  0.0211856   0.0324162   0.00213878   0.0377156   0.0683502   0.030081      -0.00684333  -0.035509   0.304867   0.26033    0.113802    0.152433  0.0

In [55]:
JuliaFEM.xdmf.xdmf_new_field(grid, "Displacement", "nodes", u)

true

In [56]:
JuliaFEM.xdmf.xdmf_save_model(xdoc, "/tmp/3d_solid_model.xmf")

28330